# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

TypeError: only size-1 arrays can be converted to Python scalars

In [3]:
from spock import CollisionMergerClassifier
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)

print('Probability planet 1 collides with 2:', prob_12)
print('Probability planet 2 collides with 3:', prob_23)
print('Probability planet 1 collides with 3:', prob_13)

Probability planet 1 collides with 2: 0.27385116
Probability planet 2 collides with 3: 0.49274087
Probability planet 1 collides with 3: 0.23340799


In [4]:
from spock import CollisionOrbitalOutcomeRegressor
reg_model = CollisionOrbitalOutcomeRegressor()

new_sim = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3])

print('Predicted P1:', new_sim.particles[1].P)
print('Predicted e1:', new_sim.particles[1].e, '\n')
print('Predicted P2:', new_sim.particles[2].P)
print('Predicted e2:', new_sim.particles[2].e)

Predicted P1: 1.2250393057947184
Predicted e1: 0.05889093054898333 

Predicted P2: 1.4126459270205634
Predicted e2: 0.029298967242683358


# Giant impact emulator

In [1]:
import rebound
from spock import GiantImpactPhaseEmulator, CollisionOrbitalOutcomeRegressor, CollisionMergerClassifier

In [9]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1, e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()
tmax = 1e9*sim.particles[1].P

In [10]:
emulator = GiantImpactPhaseEmulator(seed=0)

In [4]:
sim.angular_momentum()

<rebound.vectors.Vec3d object at 0x155964220, [0.0, 0.0, 3.1770947579198276e-05]>

In [5]:
sims = [sim]

In [8]:
sims = emulator.step(sims, tmaxs=tmax)
ps = sims[0].particles
print('t =', sims[0].t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted P' + str(i), '=', ps[i].P)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

999999999.9999993
t = 999999999.9999993 

Predicted m1 = 2e-05
Predicted P1 = 1.1024217143066415
Predicted e1 = 0.03653730622614261 

Predicted m2 = 1e-05
Predicted P2 = 1.6759667991517082
Predicted e2 = 0.05066525539270971 

Predicted m3 = 2e-05
Predicted P3 = 2.4601645924432147
Predicted e3 = 0.025964998946982995 



In [11]:
%%time
sims = emulator.predict(sim, tmaxs=tmax)

1086.8161353481798
16050087.493189935
999999999.9999993
CPU times: user 1.31 s, sys: 674 ms, total: 1.98 s
Wall time: 1.09 s


In [12]:
ps = sims[0].particles
print('t =', sims[0].t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted P' + str(i), '=', ps[i].P)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

t = 999999999.9999993 

Predicted m1 = 2e-05
Predicted P1 = 1.1024217143066415
Predicted e1 = 0.03653730622614261 

Predicted m2 = 1e-05
Predicted P2 = 1.6759667991517082
Predicted e2 = 0.05066525539270971 

Predicted m3 = 2e-05
Predicted P3 = 2.4601645924432147
Predicted e3 = 0.025964998946982995 



In [54]:
sims[0].angular_momentum()

<rebound.vectors.Vec3d object at 0x1636954b0, [-5.6352403401004125e-08, -1.1685649329778104e-07, 3.488974345271752e-05]>